# CartPole-v0 by tensorflow

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. 

A `reward` of +1 is provided for every timestep that the pole remains upright. The episode ends when `the pole is more than 15 degrees from vertical`, or `the cart moves more than 2.4 units from the center`.

In [1]:
import gym
from collections import deque
import itertools
import numpy as np
import random
import tensorflow as tf

random.seed(42)

In [2]:
class DQNAgent():
    def __init__(self, state_size, action_size):
        self._state_size = state_size
        self._action_size = action_size        
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self._learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        
        # tensorflow
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    
    def _build_model(my, hidden_size=24):
        class DQNModel():
            def __init__(self):
                self.x = tf.placeholder(tf.float32, [None, my._state_size])
                nn = tf.layers.dense(self.x, hidden_size, activation=tf.nn.relu)
                nn = tf.layers.dense(nn, hidden_size, activation=tf.nn.relu)
                self.Q = tf.layers.dense(nn, my._action_size)#, activation=tf.nn.linear)
        
                # Loss
                self.targetQ = tf.placeholder(tf.float32, [None, my._action_size])
                self.loss = tf.reduce_mean((self.Q - self.targetQ)**2)
                
                # Summary for TensorBoard
                tf.summary.scalar('loss', self.loss)
                self.summary = tf.summary.merge_all()    # summary

                # Training
                self.train_op = tf.train.AdamOptimizer(my._learning_rate).minimize(self.loss)
        
        return DQNModel()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def update_target_model(self):
        self.target_model = self.model
        
    def train_model(self, batch_size=20):
        def batch(iterable, n=1):
            l = len(iterable)
            for ndx in range(0, l, n):
                yield list(itertools.islice(iterable, ndx, min(ndx + n, l)))
        
        batch_data = list(batch(self.memory, batch_size))
        selected_batch = random.sample(batch_data, 1)
        
        for state, action, reward, next_state, done in selected_batch[0]:
            target = self.sess.run(self.model.Q, feed_dict={self.model.x: [state]})
            target_val = self.sess.run(self.model.Q, feed_dict={self.model.x: [next_state]})
            target_val_ = self.sess.run(self.target_model.Q, feed_dict={self.target_model.x: [next_state]})
            if done:
                target[0][action] = reward
            else:
                a = np.argmax(target_val)
                target[0][action] = reward + self.gamma * target_val_[0][a]
            
            feed_dict = {
                self.model.x: [state], 
                self.model.targetQ: target,
            }
            _, loss = self.sess.run([self.model.train_op, self.model.loss], feed_dict=feed_dict)
            summary = self.sess.run([self.model.summary], feed_dict=feed_dict)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay    

    def takeAction(self, state):
        q_values = self.sess.run(self.model.Q, feed_dict={self.model.x: [state]})
        action = q_values.argmax()
        if np.random.rand() < self.epsilon:
            action = random.randrange(self._action_size)
        return action


In [3]:
episodes = 500
render = True
shows = 20
total_reward = list()

env = gym.make('CartPole-v0')
state = env.reset()
agent = DQNAgent(state.shape[0], env.action_space.n)

for e in range(episodes):
    state = env.reset()
    
    for time_t in range(500):
        
        if render:
            env.render()
        
        action = agent.takeAction(state)
        next_state, reward, done, _ = env.step(action)
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            total_reward.append(time_t)
            break
            
    agent.train_model(32)
    
    if e % shows == 0:
        print("Episode: {}, Total Reward: {}".format(e, np.mean(total_reward[-shows:])))
        agent.update_target_model()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode: 0, Total Reward: 9.0
Episode: 20, Total Reward: 22.9
Episode: 40, Total Reward: 18.5
Episode: 60, Total Reward: 16.95
Episode: 80, Total Reward: 27.0
Episode: 100, Total Reward: 32.3
Episode: 120, Total Reward: 50.6
Episode: 140, Total Reward: 27.4
Episode: 160, Total Reward: 47.95
Episode: 180, Total Reward: 43.05
Episode: 200, Total Reward: 27.1
Episode: 220, Total Reward: 42.3
Episode: 240, Total Reward: 48.8
Episode: 260, Total Reward: 38.7
Episode: 280, Total Reward: 58.85
Episode: 300, Total Reward: 64.45
Episode: 320, Total Reward: 57.7
Episode: 340, Total Reward: 73.0
Episode: 360, Total Reward: 61.45
Episode: 380, Total Reward: 130.7
Episode: 400, Total Reward: 190.95
Episode: 420, Total Reward: 196.75
Episode: 440, Total Reward: 174.65
Episode: 460, Total Reward: 179.85
Episode: 480, Total Reward: 188.8
